In [ ]:
import warnings
warnings.filterwarnings("ignore")
from fragile.atari.swarm import MontezumaSwarm
import ray
ray.init()

In [ ]:
swarm = MontezumaSwarm.create_swarm(n_walkers=75,
                                    max_iters=1000,
                                    prune_tree=True,
                                    use_tree=False,
                                    reward_scale=4,
                                    dist_scale=0.5,
                                    plot_step=2,
                                    critic_scale=2,
                                   episodic_rewad=True)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
dmap = swarm.plot_dmap()
dmap

In [ ]:
_ = swarm.run_swarm(print_every=1)

In [ ]:
swarm.walkers.env_states.states.dtype

In [ ]:
import holoviews as hv
hv.extension("matplotlib")
dmap[0].opts(normalize=True)

In [ ]:
for x in swarm.walkers.env_states.states.reshape(swarm.walkers.n, -1)[0]:
    print(x)

In [ ]:
import numpy as np
np.prod((210,160,3))

In [ ]:
swarm.plot_critic()

In [ ]:
import time
import numpy as np
import pandas as pd
import holoviews as hv
import streamz
import streamz.dataframe

from holoviews import opts
from holoviews.streams import Pipe, Buffer

hv.extension('bokeh')

In [ ]:
hv.Image(data)

In [ ]:
pipe = Pipe(data=[])
vector_dmap = hv.DynamicMap(hv.Image, streams=[pipe])
vector_dmap.opts(xlim=(-0.5, 0.5), ylim=(-0.5, 0.5))

In [ ]:
x,y  = np.mgrid[-10:11,-10:11] * 0.1
sine_rings  = np.sin(x**2+y**2)*np.pi+np.pi
exp_falloff = 1/np.exp((x**2+y**2)/8)

for i in range(10):
    time.sleep(0.1)
    data = swarm.walkers.env_states.observs[i, :-3].reshape((210, 160, 3))
    pipe.send(data)

In [ ]:
import numpy as np
from plangym import ParallelEnvironment
from plangym.minimal_montezuma import Montezuma
#from plangym.montezuma import Montezuma

from fragile.core.env import DiscreteEnv
from fragile.core.dt_sampler import GaussianDt
from fragile.core.models import RandomDiscrete
from fragile.core.states import States
from fragile.core.swarm import Swarm
from fragile.core.walkers import Walkers
from fragile.core.tree import HistoryTree
from fragile.atari.walkers import MontezumaWalkers
from fragile.atari.critics import MontezumaGrid
env = ParallelEnvironment(
        env_class=Montezuma,
        name=None,
        autoreset=True,
        blocking=False,
    episodic_live=True,
    min_dt=1,
    )
dt = GaussianDt(min_dt=3, max_dt=1000, loc_dt=4, scale_dt=2)

swarm = Swarm(
    model=lambda x: RandomDiscrete(x, dt_sampler=dt),
    walkers=MontezumaWalkers,
    env=lambda: DiscreteEnv(env),
    n_walkers=50,
    max_iters=25,
    prune_tree=True,
    reward_scale=2,
    dist_scale=4,
    tree=HistoryTree,
    critic=MontezumaGrid(),
    
)



In [ ]:
from holoviews.streams import Pipe, Buffer
from streamz.dataframe import DataFrame
from streamz import Stream
import holoviews as hv
import hvplot.pandas
import hvplot.streamz
import numpy as np
import pandas as pd
hv.extension("bokeh")

In [ ]:
mm = Montezuma()

In [ ]:
o = mm.env.reset()

In [ ]:
o

In [ ]:
plt.imshow(o[50:, :, 0])

In [ ]:
from PIL import Image
def resize_frame(frame: np.ndarray, height: int, width: int, mode="RGB") -> np.ndarray:
    """
    Use PIL to resize an RGB frame to an specified height and width.

    Args:
        frame: Target numpy array representing the image that will be resized.
        height: Height of the resized image.
        width: Width of the resized image.

    Returns:
        The resized frame that matches the provided width and height.
    """
    frame = Image.fromarray(frame)
    frame = frame.convert(mode).resize((height, width))
    return np.array(frame)

In [ ]:
#def plot_memory(raw_observ, )
background = o[50:, :, ].mean(axis=2).astype(bool)
peste = resize_frame(df.values.T[::1, ::1], 160, 160, "L")
peste = peste / peste.max()# * 255
hv.Image(background)*hv.Image(peste).opts(alpha=0.7)

In [ ]:
background.astype(int).tostring()

In [ ]:
(hv.Image(o).opts(norm={'axiswise': True})+hv.Raster(df.values.T[::1, ::1]).opts(alpha=0.5, normalize=True)).opts(norm={'axiswise': True})

In [ ]:
df = swarm.critic.buffer_df

In [ ]:
df.hvplot(kind="heatmap")

In [ ]:
swarm.critic.plot_grid()

In [ ]:
df = pd.DataFrame(swarm.critic.memory[:, :, 0],
                                      columns=swarm.critic._cols, index=swarm.critic._index)

In [ ]:
_ = swarm.run_swarm(print_every=10)

In [ ]:
x = np.zeros((5,5))
x[[1,2], [0, 2]] = x[[1,2], [0, 2]] + 1
x

In [ ]:
from plangym.montezuma import MyMontezuma

In [ ]:
mm = MyMontezuma()

In [ ]:
o = mm.env.reset()

In [ ]:
mm.get_face_pixels(o)

In [ ]:
np.where(o[:, :, 0] == 228)

In [ ]:
face_pixels = [(y, x * mm.x_repeat) for y, x in mm.get_face_pixels(o)]
np.mean(face_pixels,axis=0)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(o[50:, :, 0] == 228)

In [ ]:
o[50:, :, 0].shape

In [ ]:
x = swarm.walkers.env_states.observs[:, -3]
y = swarm.walkers.env_states.observs[:, -2]
rooms = swarm.walkers.env_states.observs[:, -1]
rooms

In [ ]:
grid = np.arange(320*160).reshape((320, 160, 1))

In [ ]:
grid[x.astype(int), y.astype(int), rooms.astype(int)]

In [ ]:
np.mean([78, 80, 81, 78, 79, 80, 81, 79, 80, 81])

In [ ]:
for i in range(3):
    mm.step(mm.env.action_space.sample())
    mm.render()
    print(mm.pos)

In [ ]:
old = {1,2,3,4,5}
new = {1,3,5}

In [ ]:
old-new

In [ ]:
env.pos

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(obs[25:185,:,])

In [ ]:
15 *15

In [ ]:
from PIL import Image
def resize_frame(frame: np.ndarray, height: int, width: int, mode="RGB") -> np.ndarray:
    """
    Use PIL to resize an RGB frame to an specified height and width.

    Args:
        frame: Target numpy array representing the image that will be resized.
        height: Height of the resized image.
        width: Width of the resized image.

    Returns:
        The resized frame that matches the provided width and height.
    """
    frame = Image.fromarray(frame)
    frame = frame.convert(mode).resize((height, width))
    return np.array(frame)

In [ ]:
plt.imshow(resize_frame(obs[25:185,:], width=45, height=45, mode="L"))

In [ ]:
swarm.run_swarm?

In [ ]:
state, obs = env.reset()

states = [state.copy() for _ in range(10)]
actions = [env.action_space.sample() for _ in range(10)]

data = env.step_batch(states=states, actions=actions)
new_states, observs, rewards, ends, infos = data